## 1. Import Library

Berikut ini beberapa Library yang akan dipakai untuk melakukan analisa ke depannya.

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
!pip install emoji -q

import pandas as pd
import numpy as np
import re
import string
import emoji
import gensim


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## 2. Data Exploration

Dataset yang saya miliki berada dalam format xlsx sehingga untuk membaca format tersebut bisa menggunakan pd.read_excel

In [ ]:
dataset = pd.read_excel('/content/dataset.xlsx')

Berikut ini merupakan Top 5 dari dataset yang saya miliki

In [ ]:
dataset.head()

,Teks,Media,Label
0,Komisi Pemberantasan Korupsi (KPK) memeriksa A...,Liputan 6,Politik
1,Arsenal tidak bisa memetik poin maksimal di be...,CNN Indonesia,Olahraga
2,Polda Sumatera Utara (Sumut) membeberkan awal ...,Detik,Politik
3,sukses dilaksanakan pada tanggal 1-2 April 202...,Liputan 6,Hiburan
4,Timnas Indonesia U-22 telah mendarat di Kamboj...,Detik,Olahraga


Berikut ini merupakan 5 data terakhir dari dataset yang saya miliki.

In [ ]:
dataset.tail()

,Teks,Media,Label
5,Komisi Pemberantasan Korupsi (KPK) memeriksa m...,Kompas,Politik
6,Kepala Dinas Pariwisata dan Ekonomi Kreatif (P...,Kompas,Hiburan
7,Warner Bros. mengumumkan game terbaru dari sem...,Liputan 6,Olahraga
8,Wali Kota Solo Kota Solo Gibran Rakabuming Rak...,Detik,Hiburan
9,Presiden Joko Widodo atau Jokowi disebut akan ...,Liputan 6,Politik


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Teks    10 non-null     object
 1   Media   10 non-null     object
 2   Label   10 non-null     object
dtypes: object(3)
memory usage: 368.0+ bytes


Output di atas menunjukkan bahwa terdapat sebanyak 3 column di dataset yang saya miliki yaitu "Teks, Media dan Label" dimana masing - masing column tersebut memiliki data sebanyak 10 baris. Dimana data type berupa object.

## 3. Data Preparation / Data Cleaning

### 3.1 Remove Emoji, HTML and Stop Words

In [ ]:
def remove_html(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)

def remove_emoji(x):
    return emoji.replace_emoji(x, '')

def cleaning(dataset, stop_words):
  # Menghapus tag html
    dataset['Teks'] = dataset['Teks'].apply(lambda x: remove_html(x))

    # Menghapus emoji (jika ada)
    dataset['Teks'] = dataset['Teks'].apply(lambda x: remove_emoji(x))

    # Removing stop words
    dataset['Teks'] = dataset['Teks'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop_words))

    return dataset

stop_words = stopwords.words('indonesian')
clean_text = cleaning(dataset, stop_words)

Code di atas itu berguna untuk menghapus emoji, serta link HTML

Function di atas juga menghapus stopword di dataset yang saya miliki, karena saya mengambil berita yang berasal dari dalam negri maka stopword yang saya gunakan itu 'indonesian'. Kenapa harus dilakukan tahapan menghapus stopword?? Hal ini dikarenakan stopword itu keberadannya dalam suatu kalimat ada atau tidak itu gk mempengaruhi keseluruhan/makna dari kalimat tersebut.

### 3.2 Case Folding

In [ ]:
clean_text=clean_text['Teks'].str.casefold()
clean_text

0    komisi pemberantasan korupsi (kpk) memeriksa a...
1    arsenal memetik poin maksimal laga menentukan ...
2    polda sumatera utara (sumut) membeberkan penga...
3    sukses dilaksanakan tanggal 1-2 april 2023 lan...
4    timnas indonesia u-22 mendarat kamboja. anak a...
5    komisi pemberantasan korupsi (kpk) memeriksa m...
6    kepala dinas pariwisata ekonomi kreatif (parek...
7    warner bros. mengumumkan game terbaru semesta ...
8    wali kota solo kota solo gibran rakabuming rak...
9    presiden joko widodo jokowi mengumpulkan ketua...
Name: Teks, dtype: object

Disini saya menggunakan function Case Folding dibandingkan dengan function lower, hal ini dikarenakan Case Folding itu include karakter unik yang tidak terdapat di ASCII sedangkan lower itu hanya mengubah karakter yang ada di ASCII saja. Jadi untuk jaga - jaga ada karakter unik saya menggunakan Case Folding.

### 3.3 Remove Number

In [ ]:
clean_text=[re.sub(r"\d+","",i )for i in clean_text]
clean_text

['komisi pemberantasan korupsi (kpk) memeriksa anggota dprd dki jakarta bernama cinta mega. pemeriksaan cinta terkait dugaan korupsi pengadaan tanah kelurahan pulogebang, cakung, jakarta timur -. diketahui, cinta anggota dprd dki jakarta fraksi pdi perjuangan. saat cinta hadir memenuhi panggilan kpk kapasitasnya saksi."pemeriksaan komisi pemberantasan korupsi, jl. kuningan persada kav-, setiabudi, kuningan, jakarta selatan, nama hj. cinta mega s.h. anggota dprd provinsi dki jakarta periode -," kepala bagian pemberitaan kpk ali fikri keterangan diterima, rabu (//). menurut ali, keterangan cinta pemeriksaan dianggap menyelesaikan berita acara pemeriksaan (bap) dugaan korupsi pengadaan tanah kelurahan pulogebang. ini kali cinta memenuhi panggilan kpk saksi. pada lalu, cinta hadir diberondong terkait pembahasan anggaran pengadaan tanah pulogebang diperuntukan program rumah dp rp . sebagai informasi, kpk mengungkap dugaan tindak pidana korupsi pengadaan tanah kelurahan pulogebang, cakung, j

Disini saya menghapus angka yang terdapat di dalam kalimat yang sedang saya amati. Lebih tepatnya mengganti angka tersebut menjadi sebuah string kosong.

### 3.4 Remove Punctuation

Selanjutnya akan dilakukan penghapusan tanda baca yang terdapat dalam kalimat. Jadi akan ada beberapa tanda baca yang dihapus.

In [ ]:
clean_text=[re.sub(r'[\W]',' ',i )for i in clean_text]
clean_text

['komisi pemberantasan korupsi  kpk  memeriksa anggota dprd dki jakarta bernama cinta mega  pemeriksaan cinta terkait dugaan korupsi pengadaan tanah kelurahan pulogebang  cakung  jakarta timur    diketahui  cinta anggota dprd dki jakarta fraksi pdi perjuangan  saat cinta hadir memenuhi panggilan kpk kapasitasnya saksi  pemeriksaan komisi pemberantasan korupsi  jl  kuningan persada kav   setiabudi  kuningan  jakarta selatan  nama hj  cinta mega s h  anggota dprd provinsi dki jakarta periode     kepala bagian pemberitaan kpk ali fikri keterangan diterima  rabu       menurut ali  keterangan cinta pemeriksaan dianggap menyelesaikan berita acara pemeriksaan  bap  dugaan korupsi pengadaan tanah kelurahan pulogebang  ini kali cinta memenuhi panggilan kpk saksi  pada lalu  cinta hadir diberondong terkait pembahasan anggaran pengadaan tanah pulogebang diperuntukan program rumah dp rp   sebagai informasi  kpk mengungkap dugaan tindak pidana korupsi pengadaan tanah kelurahan pulogebang  cakung  j

Nah, disini sudah tidak terdapat tanda baca apapun dalam kalimat.

### 3.5 Remove Multiple Whitespace

In [ ]:
clean_text=[re.sub(r'\s+',' ',i )for i in clean_text]
clean_text

['komisi pemberantasan korupsi kpk memeriksa anggota dprd dki jakarta bernama cinta mega pemeriksaan cinta terkait dugaan korupsi pengadaan tanah kelurahan pulogebang cakung jakarta timur diketahui cinta anggota dprd dki jakarta fraksi pdi perjuangan saat cinta hadir memenuhi panggilan kpk kapasitasnya saksi pemeriksaan komisi pemberantasan korupsi jl kuningan persada kav setiabudi kuningan jakarta selatan nama hj cinta mega s h anggota dprd provinsi dki jakarta periode kepala bagian pemberitaan kpk ali fikri keterangan diterima rabu menurut ali keterangan cinta pemeriksaan dianggap menyelesaikan berita acara pemeriksaan bap dugaan korupsi pengadaan tanah kelurahan pulogebang ini kali cinta memenuhi panggilan kpk saksi pada lalu cinta hadir diberondong terkait pembahasan anggaran pengadaan tanah pulogebang diperuntukan program rumah dp rp sebagai informasi kpk mengungkap dugaan tindak pidana korupsi pengadaan tanah kelurahan pulogebang cakung jakarta timur perusahaan umum daerah perumd

Disini dilakukan penghapusan apabila terdapat double space (dua kali spasi/string kosong), sebelumnya kan kita sudah replace number menjadi sebuah string kosong jadi otomatis nanti ada kasus double space. Disinilah hal itu ditangani.

### 3.6 Tokenizer

In [ ]:
word_token = [word_tokenize(i) for i in clean_text]
word_token

[['komisi',
  'pemberantasan',
  'korupsi',
  'kpk',
  'memeriksa',
  'anggota',
  'dprd',
  'dki',
  'jakarta',
  'bernama',
  'cinta',
  'mega',
  'pemeriksaan',
  'cinta',
  'terkait',
  'dugaan',
  'korupsi',
  'pengadaan',
  'tanah',
  'kelurahan',
  'pulogebang',
  'cakung',
  'jakarta',
  'timur',
  'diketahui',
  'cinta',
  'anggota',
  'dprd',
  'dki',
  'jakarta',
  'fraksi',
  'pdi',
  'perjuangan',
  'saat',
  'cinta',
  'hadir',
  'memenuhi',
  'panggilan',
  'kpk',
  'kapasitasnya',
  'saksi',
  'pemeriksaan',
  'komisi',
  'pemberantasan',
  'korupsi',
  'jl',
  'kuningan',
  'persada',
  'kav',
  'setiabudi',
  'kuningan',
  'jakarta',
  'selatan',
  'nama',
  'hj',
  'cinta',
  'mega',
  's',
  'h',
  'anggota',
  'dprd',
  'provinsi',
  'dki',
  'jakarta',
  'periode',
  'kepala',
  'bagian',
  'pemberitaan',
  'kpk',
  'ali',
  'fikri',
  'keterangan',
  'diterima',
  'rabu',
  'menurut',
  'ali',
  'keterangan',
  'cinta',
  'pemeriksaan',
  'dianggap',
  'menyelesa

Output di atas merupakan hasil tokenize dari data yang telah dibersihkan sebelumnya. Jadi tokenize ini merupakan proses untuk membagi teks yang berupa sebuah kalimat, paragraf, dokumen dan lain-lain ke dalam bentuk token/bagian tertentu. Jadi dapat dilihat bahwa setiap data memiliki tokennya masing-masing dan berbeda satu sama lain.

### Word2Vec - CBOW

Continuous Bag of Words(CBOW) merupakan sebuah arsitektur yang memprediksi kata yang tepat berdasarkan konteksnya (konteks disini bisa berupa satu kata atau beberapa kata yang muncul sebelum dan sesudah kata yang ingin diprediksi)

In [ ]:
model_cbow = gensim.models.Word2Vec(word_token, min_count=2, vector_size=50, window=5, sg=0) #sg=0 ini algoritma yg digunakan

Input untuk function CBOW ini merupakan kata setelah di tokenize, makanya pada code di atas itu input berupa (word_token)

min_count itu maksudnya berapa jumlah minimal frekuensi yang akan diperhitungkan sebagai token untuk di ubah ke vektor. Jadi kata yang diperhitungkan itu jika kemunculannya dalam kalimat itu sebanyak 2kali. Misalkan kata "komisi" itu muncul sebanyak 3kali maka itu diperhitungkan, jika muncul hanya sekali maka itu tidak diperhitungkan.

vector_size ini memperlihatkan berapa banyak neuron yang di set di hidden layer. Jadi saya kan pakai size 50, maka vector saya dimensinya itu 50. Jadi satu kata direpresentasikan ke dalam 50 vektor

window = 5, itu satu konteks terdiri dari 4 kata

sg-0 itu merupakan algoritma yang dipakai untuk metode CBOW

In [ ]:
vocabulary_cbow = model_cbow.wv.key_to_index
print(vocabulary_cbow)

{'jakarta': 0, 'hiburan': 1, 'kpk': 2, 'arsenal': 3, 'cinta': 4, 'dki': 5, 'korupsi': 6, 'city': 7, 'ini': 8, 'pengadaan': 9, 'game': 10, 'ibas': 11, 'pemeriksaan': 12, 'rabu': 13, 'gibran': 14, 'koko': 15, 'quidditch': 16, 'ketum': 17, 'ali': 18, 'baju': 19, 'tanah': 20, 'teman': 21, 'potter': 22, 'harry': 23, 'aditya': 24, 'ken': 25, 'indonesia': 26, 'pertemuan': 27, 'timur': 28, 'partai': 29, 'ketua': 30, 'bertemu': 31, 'poin': 32, 'pelapor': 33, 'lawan': 34, 'sarana': 35, 'jaya': 36, 'kamboja': 37, 'bros': 38, 'digelar': 39, 'warner': 40, 'pak': 41, 'manchester': 42, 'laga': 43, 'perumda': 44, 'man': 45, 'champions': 46, 'cakung': 47, 'terkait': 48, 'mengikuti': 49, 'kelurahan': 50, 'dprd': 51, 'lahan': 52, 'pemprov': 53, 'wanita': 54, 'kali': 55, 'sumut': 56, 'pan': 57, 'itu': 58, 'duel': 59, 'tanggal': 60, 'mengatakan': 61, 'perusahaan': 62, 'penonton': 63, 'politik': 64, 'umum': 65, 'buka': 66, 'anggota': 67, 'bareng': 68, 'zulhas': 69, 'membuka': 70, 'selasa': 71, 'polda': 72, 

Output di atas merupakan list vocabulary yang telah dihasilkan dari model_cbow sebelumnya. Vocabulary ini berisi seluruh token yang kemunculannya sebanyak 2 atau lebih dari 2kali, serta merupakan token dari hasil target

In [ ]:
similar_words = model_cbow.wv.most_similar("jakarta")
print(similar_words)

[('kondisi', 0.3368767201900482), ('menurut', 0.32799091935157776), ('situs', 0.31143882870674133), ('bila', 0.2985489070415497), ('ali', 0.2895926237106323), ('sumut', 0.27857160568237305), ('utama', 0.2734213173389435), ('kepala', 0.27007997035980225), ('laga', 0.2687310576438904), ('poin', 0.2616409659385681)]


Berikut ini merupakan prediksi kata yang mirip dengan kata "jakarta" dimana dapat dilihat setiap kata yang dianggap mirip itu memiliki probability masing - masing, dan hal ini bisa dibilang cukup rendah jadi dapat disimpulkan bahwa kata "kondisi" itu merupakan yang paling mirip dengan kata "jakarta" dengan probability sekitar 0.33

In [ ]:
#get the vector
vectors_cbow = [model_cbow.wv.get_vector(word)for word in vocabulary_cbow]
vectors_cbow[0]

array([-0.00029096,  0.00053419,  0.01027565,  0.01745932, -0.01802573,
       -0.01478087,  0.01348873,  0.01974768, -0.01133448, -0.00754075,
        0.01456883, -0.00494906, -0.00862506,  0.01418595, -0.009557  ,
       -0.00283797,  0.00661964,  0.00251423, -0.01783533, -0.01899626,
        0.0148884 ,  0.01086648,  0.01422051,  0.00118222,  0.01379811,
       -0.00682047, -0.0019861 ,  0.01087314, -0.01570456, -0.00724179,
       -0.01500809, -0.00145872,  0.01804235, -0.01477729, -0.00586692,
       -0.00274164,  0.01800805, -0.01204315,  0.00138932, -0.01033066,
       -0.01784864,  0.01023483, -0.01751214, -0.00900264,  0.00130407,
       -0.00029437, -0.01573473,  0.01928978,  0.01032674,  0.01846505],
      dtype=float32)

Output di atas merupakan representasi dari token 0 yaitu kata "jakarta" yang di representasikan ke dala 50 vektor CBOW

### Word2Vec - Skipgram

Arsitektur Skipgram itu memprediksi konteks yang tepat berdasarkan kata targetnya. Artinya, model belajar memprediksi kemunculan kata - kata tertentu berdasarkan kemunculan kata target yang diberikan

In [ ]:
model_skipgram = gensim.models.Word2Vec(word_token, min_count = 2,vector_size = 50, window = 5, sg=1)

Secara keseluruhan function yang digunakan itu cenderung sama, perbedannya hanya terletak pada algoritma yang digunakan, dimana pada model Skipgram itu dipakai (sg=1)

In [ ]:
vocabulary_skipgram = model_skipgram.wv.key_to_index
print(vocabulary_skipgram)

{'jakarta': 0, 'hiburan': 1, 'kpk': 2, 'arsenal': 3, 'cinta': 4, 'dki': 5, 'korupsi': 6, 'city': 7, 'ini': 8, 'pengadaan': 9, 'game': 10, 'ibas': 11, 'pemeriksaan': 12, 'rabu': 13, 'gibran': 14, 'koko': 15, 'quidditch': 16, 'ketum': 17, 'ali': 18, 'baju': 19, 'tanah': 20, 'teman': 21, 'potter': 22, 'harry': 23, 'aditya': 24, 'ken': 25, 'indonesia': 26, 'pertemuan': 27, 'timur': 28, 'partai': 29, 'ketua': 30, 'bertemu': 31, 'poin': 32, 'pelapor': 33, 'lawan': 34, 'sarana': 35, 'jaya': 36, 'kamboja': 37, 'bros': 38, 'digelar': 39, 'warner': 40, 'pak': 41, 'manchester': 42, 'laga': 43, 'perumda': 44, 'man': 45, 'champions': 46, 'cakung': 47, 'terkait': 48, 'mengikuti': 49, 'kelurahan': 50, 'dprd': 51, 'lahan': 52, 'pemprov': 53, 'wanita': 54, 'kali': 55, 'sumut': 56, 'pan': 57, 'itu': 58, 'duel': 59, 'tanggal': 60, 'mengatakan': 61, 'perusahaan': 62, 'penonton': 63, 'politik': 64, 'umum': 65, 'buka': 66, 'anggota': 67, 'bareng': 68, 'zulhas': 69, 'membuka': 70, 'selasa': 71, 'polda': 72, 

In [ ]:
similar_words = model_skipgram.wv.most_similar("jakarta")
print(similar_words)

[('bila', 0.5168485641479492), ('gibran', 0.5018904209136963), ('ali', 0.4883163869380951), ('utama', 0.4785524904727936), ('kondisi', 0.47130444645881653), ('partai', 0.4595808684825897), ('poin', 0.4594932198524475), ('laga', 0.45080000162124634), ('situs', 0.4429027736186981), ('harry', 0.4425364136695862)]


Berikut ini merupakan prediksi kata yang mirip dengan kata "jakarta" dimana dapat dilihat setiap kata yang dianggap mirip itu memiliki probability masing - masing, dan hal ini bisa dibilang lebih bagus daripada model CBOW sebelumnya.jadi dapat disimpulkan bahwa kata "bila" itu merupakan yang paling mirip dengan kata "jakarta" dengan probability sekitar 0.51

In [ ]:
vectors_skipgram = [model_skipgram.wv.get_vector(word)for word in vocabulary_skipgram]
vectors_skipgram[0]

array([ 0.00832586,  0.00047671,  0.00950214,  0.01415935, -0.01214896,
       -0.02101981,  0.02119086,  0.03543984, -0.02161324, -0.00775715,
        0.01262646, -0.01958074, -0.00616778,  0.02380563, -0.01013242,
        0.00348633,  0.00949014,  0.0042235 , -0.02696216, -0.02097323,
        0.01475663,  0.0176289 ,  0.0194933 , -0.00274544,  0.02270643,
       -0.00828574, -0.00333926,  0.00770559, -0.02049436, -0.00359592,
       -0.01599584,  0.00195136,  0.01232677, -0.01679128, -0.01576879,
        0.0056144 ,  0.03186002, -0.01222448,  0.00934147, -0.02024982,
       -0.00896947,  0.01311539, -0.01854748, -0.01008014,  0.01301102,
        0.00198339, -0.02008568,  0.01785319,  0.01319746,  0.02032049],
      dtype=float32)

Output di atas merupakan representasi dari token 0 yaitu kata "jakarta" yang di representasikan ke dala 50 vektor Skipgram

### TF - IDF

In [ ]:
# Inisialisasi function
vectorizer = TfidfVectorizer()

# Menghitung matriks TF-IDF
tfidf_matrix = vectorizer.fit_transform(clean_text)

# Membuat dataframe dari matriks TF-IDF
TFIDF = pd.DataFrame(tfidf_matrix.todense(), columns=vectorizer.get_feature_names_out())

# Mencari top 5
for i, row in TFIDF.iterrows():
    top_5 = row.nlargest(5)
    print(f"Top 5 TF-IDF untuk dataset ke {i+1}:")
    print(top_5)
    print("\n")

Top 5 TF-IDF untuk dataset ke 1:
cinta         0.403422
korupsi       0.252139
kpk           0.252139
jakarta       0.246580
pulogebang    0.237282
Name: 0, dtype: float64


Top 5 TF-IDF untuk dataset ke 2:
arsenal    0.485301
city       0.441182
laga       0.220591
lawan      0.220591
man        0.220591
Name: 1, dtype: float64


Top 5 TF-IDF untuk dataset ke 3:
aditya     0.343275
ken        0.343275
pelapor    0.286062
polda      0.228850
sumut      0.228850
Name: 2, dtype: float64


Top 5 TF-IDF untuk dataset ke 4:
penonton     0.349527
berhasil     0.262145
kapanlagi    0.262145
acara        0.222847
bareng       0.222847
Name: 3, dtype: float64


Top 5 TF-IDF untuk dataset ke 5:
kamboja      0.419150
mendarat     0.251490
timnas       0.251490
indonesia    0.249388
airport      0.167660
Name: 4, dtype: float64


Top 5 TF-IDF untuk dataset ke 6:
kpk          0.339876
ali          0.242768
korupsi      0.242768
pengadaan    0.242768
jakarta      0.237416
Name: 5, dtype: float64


T

Output di atas menunjukkan top 5 kata yang sering muncul di dalam masing - masing paragraf tersebut. Semakin sering sebuah kata muncul dalam sebuah dokumen/data maka semakin penting kata tersebut di dalam dokumen/data.

Referensi Berita:

https://www.liputan6.com/news/read/5270327/korupsi-pengadaan-tanah-pulogebang-kpk-periksa-anggota-dprd-jakarta-cinta-mega

https://www.cnnindonesia.com/olahraga/20230426103159-142-942137/man-city-vs-arsenal-nasib-juara-the-gunners-masih-di-tangan-sendiri

https://news.detik.com/berita/d-6690567/anak-polisi-aniaya-mahasiswa-dipicu-chat-soal-wanita-inisial-d

https://www.liputan6.com/showbiz/read/5253096/kapanlagi-buka-bareng-2023-sukses-sajikan-hiburan-musik-dan-euforia-ramadhan-seru

https://sport.detik.com/sepakbola/liga-indonesia/d-6689535/timnas-indonesia-u-22-tiba-di-kamboja-arya-ayo-rebut-medali-emas

https://nasional.kompas.com/read/2023/04/26/13300511/kpk-periksa-eks-anggota-dprd-dki-cinta-mega-terkait-korupsi-pengadaan-lahan

https://megapolitan.kompas.com/read/2020/07/21/16465251/meski-didemo-pemprov-dki-belum-akan-buka-tempat-hiburan-malam

https://www.liputan6.com/tekno/read/5268122/siap-siap-para-fans-harry-potter-bakal-ada-game-baru-yang-angkat-olahraga-sihir-quidditch

https://news.detik.com/pemilu/d-6690514/ngefans-gibran-kini-cuma-follow-ibas-yudhoyono-di-twitter

https://www.liputan6.com/news/read/5270371/jokowi-akan-kumpulkan-ketum-parpol-koalisi-pemerintah-bahas-situasi-politik-terkini